In [142]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
# import csv

In [143]:
# import word list
with open('data/allowed_words.txt','r') as f:
    long_word_list = [word.strip() for word in f.readlines()]
with open('data/possible_words.txt','r') as f:
    short_word_list = [word.strip() for word in f.readlines()]

# generate reply list
reply_list = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                for m in range(3):
                    reply_list.append('{:d}{:d}{:d}{:d}{:d}'.format(i, j, k, l, m))

In [144]:
# generate the map of reply of each pair
# 生成每个单词对的回复
def generate_map_of_replies():
    for i,word1 in enumerate(short_word_list):
        for j,word2 in enumerate(short_word_list):
            map_of_reply[(i,j)] = reply_of_2_words(word1,word2)

# word1 is the tested word, word2 is the answer word 
def reply_of_2_words(word1, word2):
    # 1:必须在其他位置有
    # 2:必须在相同位置有
    reply = [0,0,0,0,0]
    for i in range(5):
        if word1[i] == word2[i]:
            reply[i] = 2
        elif word1[i] in word2:
            reply[i] = 1
            word2 = word2.replace(word1[i],'*',1)
        else:
            reply[i] = 0
    return ''.join([str(i) for i in reply])

In [145]:
# generate the map of reply of each pair, to SAVE TIME
# this step costs about 10 seconds
map_of_reply = {}
generate_map_of_replies()

In [146]:
# # 将哈希表 map_of_reply 存储为 CSV 文件
# with open('result/map_of_reply.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['row', 'col', 'value'])
#     for key, value in map_of_reply.items():
#         writer.writerow([long_word_list[key[0]],long_word_list[key[1]], ','.join(str(x) for x in value)])

In [147]:
# game solution0: use E_info_list created before every guess
def solution0(start_word):
    # E_info_df= pd.read_csv('result/E_info_for_all_words.csv')
    if start_word == '':
        start_word = 'saner'
    guessing_word = start_word
    possible_words = short_word_list.copy()
    guessed_words = []
    turn = 0

    while True:
        if guessing_word == ANSWER:
            print('end')
            break
        turn += 1

        reply = map_of_reply[(short_word_list.index(guessing_word),short_word_list.index(ANSWER))]  

        guessed_words.append(guessing_word)
        possible_words_old = possible_words.copy()
        possible_words=[]
        for word in possible_words_old:
            if map_of_reply[(short_word_list.index(guessing_word),short_word_list.index(word))] == reply and word not in guessed_words:
                possible_words.append(word)

        print('turn:',turn,'    possible_words: ', possible_words)     

        # find out the word with max E_info in E_info_df
        max_E_info = 0
        for word in possible_words:
            E_info = E_info_df.loc[E_info_df['word'] == word]['E_info'].values[0]
            if E_info > max_E_info:
                max_E_info = E_info
                guessing_word = word

        print('turn:',turn+1,'    guessing word: ', guessing_word)
        
    return turn+1

In [148]:
# game solution1: use the word has the highest E_info satisfied the reply every time
def solution1(start_word):
    if start_word == '':
        start_word = 'saner'
    guessing_word = start_word
    possible_words = short_word_list.copy()
    guessed_words = []
    turn = 0

    while True:
        if guessing_word == ANSWER:
            print('end')
            break
        turn += 1

        reply = map_of_reply[(short_word_list.index(guessing_word),short_word_list.index(ANSWER))]  

        guessed_words.append(guessing_word)
        possible_words_old = possible_words.copy()
        possible_words=[]
        for word in possible_words_old:
            if map_of_reply[(short_word_list.index(guessing_word),short_word_list.index(word))] == reply and word not in guessed_words:
                possible_words.append(word)

        print('turn:',turn,'    possible_words: ', possible_words)     

        E_info_list = []
        for word in possible_words:
            E_info_list.append(calc_E_info(word, possible_words))
        guessing_word = possible_words[np.argmax(E_info_list)]

        print('turn:',turn+1,'    guessing word: ', guessing_word)
        
    return turn+1

# 根据（新的）可能的词库计算E_info
def calc_E_info(word, possible_words):
    E_info = 0
    # 遍历所有 word 对应的可能的 reply，去除重复的值
    for reply in list(set([value for key, value in map_of_reply.items() if key[0] == short_word_list.index(word)])):
    # for reply in reply_list:
        # 找出map_of_reply中key[0]为word,且value为reply的个数
        # 使用列表推导式和 if 语句筛选出符合条件的键值对，并使用 len() 函数计算符合条件的键值对的个数
        count = len([value for key, value in map_of_reply.items() if key[0] == short_word_list.index(word) and value == reply])
        p_x= count/len(possible_words)  
        if p_x != 0:
            E_info += p_x*np.log2(1/p_x)

        # print('word:',word,'reply:',reply,'count:',count,'p_x:',p_x)  # test code      
    return E_info


In [149]:
'''
# 计算猜的第一个词的E_info列表，根据E_info列表选择第一个猜的词
# this step costs about
# read csv if exists
if os.path.exists('result/E_info_list_1.csv'):
    E_info_list_1 =pd.read_csv('result/E_info_list_1.csv')  
else:
    E_info_list_1 = pd.DataFrame(columns=['word', 'E_info'])
    for word in tqdm(short_word_list):
        E_info_list_1.loc[len(E_info_list_1)] = [word,calc_E_info(word, short_word_list)]
    E_info_list_1.sort_values(by='E_info',ascending=False,inplace=True)
    E_info_list_1.to_csv('result/E_info_list_1.csv',index=False)
E_info_list_1.head()
'''

  0%|          | 0/2309 [00:00<?, ?it/s]

  0%|          | 0/2309 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [150]:
E_info_df= pd.read_csv('result/E_info_for_all_words.csv')

In [151]:
guess_record = []

for i in range(1000):
    ANSWER = np.random.choice(short_word_list)
    # guess_record.append({'ANSWER': ANSWER, 'turn': solution1('slate')})
    guess_record.append({'ANSWER': ANSWER, 'turn': solution0('slate')}) # simu 1000 times for about 2 minutes

guess_record_df = pd.DataFrame(guess_record)

turn: 1     possible_words:  ['screw', 'seedy', 'semen', 'serif', 'serum', 'seven', 'sever', 'sewer', 'sheen', 'sheep', 'sheer', 'sheik', 'shied', 'shrew', 'sinew', 'siren', 'skier', 'sneer', 'sober', 'sower', 'speck', 'speed', 'spend', 'sperm', 'spied', 'super', 'surer', 'sweep']
turn: 2     guessing word:  siren
turn: 2     possible_words:  ['semen', 'seven', 'sheen']
turn: 3     guessing word:  semen
turn: 3     possible_words:  ['sheen']
turn: 4     guessing word:  sheen
end
turn: 1     possible_words:  ['sadly', 'salad', 'sally', 'salon', 'salsa', 'salvo', 'shoal', 'solar']
turn: 2     guessing word:  solar
turn: 2     possible_words:  ['salad']
turn: 3     guessing word:  salad
end
turn: 1     possible_words:  ['befit', 'beget', 'beret', 'comet', 'covet', 'crept', 'debit', 'debut', 'depot', 'deter', 'detox', 'duvet', 'edict', 'egret', 'eight', 'eject', 'enter', 'entry', 'erect', 'erupt', 'ether', 'ethic', 'event', 'evict', 'exert', 'fetch', 'fetid', 'greet', 'inept', 'inert', 'in

In [152]:
# 平均猜测次数
average_turn_sol1 = guess_record_df.turn.mean()
average_turn_sol1

3.701